## Tasks (Polyglot)

In [ ]:
// // test

open testing

()



## types

In [ ]:
inl types () =
    rust.types ()
    sm'.types ()

()



## task_name

In [ ]:
nominal task_name = string

()



## manual_scheduling

In [ ]:
union manual_scheduling =
    | WithSuggestion
    | WithoutSuggestion

()



## recurrency_offset

In [ ]:
union recurrency_offset =
    | Days : i32
    | Weeks : i32
    | Months : i32

()



## day_of_week

In [ ]:
union day_of_week =
    | Sunday
    | Monday
    | Tuesday
    | Wednesday
    | Thursday
    | Friday
    | Saturday

()



## month

In [ ]:
union month =
    | January
    | February
    | March
    | April
    | May
    | June
    | July
    | August
    | September
    | October
    | November
    | December

()



## day

In [ ]:
nominal day = i32

()



## year

In [ ]:
nominal year = i32

()



## fixed_recurrency

In [ ]:
union fixed_recurrency =
    | Weekly : day_of_week
    | Monthly : day
    | Yearly : day * month

()



## recurrency

In [ ]:
union recurrency =
    | Offset : recurrency_offset
    | Fixed : list fixed_recurrency

()



## scheduling

In [ ]:
union scheduling =
    | Manual : manual_scheduling
    | Recurrent : recurrency

()



## task

In [ ]:
type task =
    {
        name : task_name
        scheduling : scheduling
    }

()



## date

In [ ]:
type date =
    {
        year : year
        month : month
        day : day
    }

()



## status

In [ ]:
union status =
    | Postponed : option ()

()



## event

In [ ]:
type event =
    {
        date : date
        status : status
    }

()



## task_template

In [ ]:
type task_template =
    {
        task : task
        events : list event
    }

()



## get_tasks (test)

In [ ]:
// // test

inl get_tasks () : list task_template =
    [
        {
            task =
                {
                    name = task_name "01"
                    scheduling = Manual WithSuggestion
                }
            events = []
        }
        {
            task =
                {
                    name = task_name "02"
                    scheduling = Manual WithSuggestion
                }
            events = []
        }
        {
            task =
                {
                    name = task_name "03"
                    scheduling = Manual WithSuggestion
                }
            events = []
        }
    ]

()



In [ ]:
// // test
// // rust=

types ()
inl tasks = get_tasks () |> sm'.format_pretty'
trace Verbose
    fun () => "Tasks"
    fun () => $'$"tasks: {!tasks} / {!_locals ()}"'

.rs output:
08:54:04 verbose #1 Tasks / tasks: UH2_1(
    UH0_0,
    "01",
    US4_0(
        US3_0,
    ),
    UH2_1(
        UH0_0,
        "02",
        US4_0(
            US3_0,
        ),
        UH2_1(
            UH0_0,
            "03",
            US4_0(
                US3_0,
            ),
            UH2_0,
        ),
    ),
)




.fsx:
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Func0<$0>")>]
#endif
type Func0<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Func1<$0, $1>")>]
#endif
type Func0<'T, 'U> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Box<$0>")>]
#endif
type Box<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("dyn $0")>]
#endif
type Dyn<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("$0 + Send")>]
#endif
type Send<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Fn() -> $0")>]
#endif
type Fn<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Fn()")>]
#endif
type FnUnit = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("FnOnce() -> $0")>]
#endif
type FnOnce<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Fn($0)")>]
#endif
type ActionFn<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Fn($0, $

In [ ]:
// // test
// // rust=

get_tasks ()
|> listm'.try_item 0i32
|> fun (Some task) => task.task.name
|> _assert_eq (task_name "01")

.rs output:
"01"




.fsx:
let rec method0 () : unit =
    let v0 : string = "01"
    let v1 : string = $"%A{v0}"
    System.Console.WriteLine v1
    let v2 : string = $"__expect / actual: %A{v0} / expected: %A{v0}"
    ()
method0()


.rs:
#![allow(dead_code)]
#![allow(non_camel_case_types)]
#![allow(non_snake_case)]
#![allow(non_upper_case_globals)]
#![allow(unreachable_code)]
#![allow(unused_attributes)]
#![allow(unused_imports)]
#![allow(unused_macros)]
#![allow(unused_parens)]
#![allow(unused_variables)]
mod module_ccfa04bf {
    pub mod Spiral_eval {
        use super::*;
        use fable_library_rust::Native_::on_startup;
        use fable_library_rust::String_::printfn;
        use fable_library_rust::String_::sprintf;
        use fable_library_rust::String_::string;
        pub fn method0() {
            let v1: string = sprintf!("{:?}", string("01"));
            printfn!("{0}", v1);
            {
                let v2: string = sprintf!(
                    "__expect / actual: {:?} / expected: 